In [1]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.8 MB/s eta 0:00:00


In [2]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.9/323.9 kB 5.0 MB/s eta 0:00:00


In [15]:
import re
import pandas as pd
import pypdf
import spacy
from IPython.display import display
from ipywidgets import FileUpload, Dropdown, VBox
from io import BytesIO # Import BytesIO

nlp = spacy.load("en_core_web_sm")

def extract_text_pdf(file):
    reader = pypdf.PdfReader(BytesIO(file)) # Wrap the bytes in BytesIO
    text = ""
    for page in reader.pages:
        page_text = page.extract_text()
        if page_text:
            text += page_text + "\n"
    return text

def parse_hdfc_statement(text):
    data = {}
    match = re.search(r'Name\s*:\s*(.+)', text)
    data['cardholder_name'] = match.group(1).strip() if match else None
    data['bank_name'] = "HDFC"
    match = re.search(r'Statement Date\s*:\s*(\d{2}/\d{2}/\d{4})', text)
    data['statement_date'] = match.group(1) if match else None

    headers_map = {
        "Payment Due Date": ["payment_due_date", "total_dues", "minimum_amount_due"],
        "Credit Limit": ["credit_limit", "available_credit", "available_cash_limit"]
    }
    lines = [line.strip() for line in text.split("\n") if line.strip()]
    i = 0
    while i < len(lines):
        line = lines[i]
        for header, fields in headers_map.items():
            if header in line:
                value_line = re.sub(r'\s+', ' ', lines[i+1])
                numbers = re.findall(r'\d{2}/\d{2}/\d{4}|[\d,]+\.\d{2}|[\d,]+', value_line)
                for field_name, num in zip(fields, numbers):
                    if '/' in num:
                        data[field_name] = num
                    elif '.' in num:
                        data[field_name] = float(num.replace(',', ''))
                    else:
                        data[field_name] = int(num.replace(',', ''))
                i += 1
        i += 1

    pattern_txn = r'(\d{2}/\d{2}/\d{4})\s+\d{2}:\d{2}:\d{2}\s+(.+?)\s+([\d,]+\.\d{2})(Cr|Dr)?'
    transactions = []
    for m in re.finditer(pattern_txn, text):
        date, desc, amount, txn_type = m.groups()
        amount = float(amount.replace(',', ''))
        txn_type = 'credit' if txn_type == 'Cr' else 'debit'
        transactions.append({'date': date, 'description': desc.strip(), 'amount': amount, 'type': txn_type})
    data['transactions'] = transactions
    return data

def parse_chase_statement(text):
    data = {}
    match = re.search(r'\n([A-Z][A-Z ]+)\n\d{1,5} [A-Z ]+ APT \d+', text)
    data['cardholder_name'] = match.group(1).strip() if match else None
    data['bank_name'] = "Chase"
    match = re.search(r'Statement Date[: ]+(\d{2}/\d{2}/\d{2,4})', text)
    if match:
        data['statement_date'] = match.group(1)
    else:
        match = re.search(r'Opening/Closing Date (\d{2}/\d{2}/\d{2,4}) - (\d{2}/\d{2}/\d{2,4})', text)
        data['statement_date'] = match.group(2) if match else None

    match = re.search(r'Account Number:\s+XXXX XXXX XXXX (\d{4})', text)
    data['account_last4'] = match.group(1) if match else None

    match = re.search(r'Payment Due Date[: ]+(\d{2}/\d{2}/\d{2,4})', text)
    data['payment_due_date'] = match.group(1) if match else None

    match = re.search(r'New Balance[: ]+\$?([\d,]+\.\d{2})', text)
    data['new_balance'] = float(match.group(1).replace(',', '')) if match else None

    match = re.search(r'Minimum Payment Due[: ]+\$?([\d,]+\.\d{2})', text)
    data['minimum_payment_due'] = float(match.group(1).replace(',', '')) if match else None

    match = re.search(r'Credit Limit[: ]+\$?([\d,]+)\s+Available Credit[: ]+\$?([\d,]+)\s+Cash Access Line[: ]+\$?([\d,]+)', text)
    if match:
        data['credit_limit'] = int(match.group(1).replace(',', ''))
        data['available_credit'] = int(match.group(2).replace(',', ''))
        data['cash_access_line'] = int(match.group(3).replace(',', ''))

    lines = text.split("\n")
    transactions = []
    txn_pattern = r'^(\d{2}/\d{2})\s+(.+?)\s+(-?\d+\.\d{2})$'
    for line in lines:
        line = line.strip()
        m = re.match(txn_pattern, line)
        if m:
            date, desc, amount = m.groups()
            amount = float(amount)
            txn_type = 'debit' if amount < 0 else 'credit'
            transactions.append({'date': date, 'description': desc.strip(), 'amount': abs(amount), 'type': txn_type})
    data['transactions'] = transactions
    return data

def main():
    uploader = FileUpload(accept='.pdf', multiple=False, description='Upload PDF Statement')
    bank_selector = Dropdown(options=['Select Bank', 'HDFC', 'Chase'], description='Select Bank:')

    def on_upload_change(change):
        if uploader.value and bank_selector.value != 'Select Bank':
            file_info = list(uploader.value.values())[0]
            content = file_info['content']
            bank = bank_selector.value.lower()
            text = extract_text_pdf(content)
            if bank == 'hdfc':
                data = parse_hdfc_statement(text)
            elif bank == 'chase':
                data = parse_chase_statement(text)
            else:
                print("Bank not supported.")
                return
            print("\nCard Info:\n")
            card_info = {k: v for k, v in data.items() if k != 'transactions'}
            display(pd.DataFrame([card_info]))
            if data['transactions']:
                print("\nTransactions:\n")
                txn_df = pd.DataFrame(data['transactions'])
                display(txn_df.head(20))

    uploader.observe(on_upload_change, names='value')
    bank_selector.observe(on_upload_change, names='value')

    display(VBox([uploader, bank_selector]))

main()


Card Info:



,cardholder_name,bank_name,statement_date,account_last4,payment_due_date,new_balance,minimum_payment_due,credit_limit,available_credit,cash_access_line
0,HARSH N JAIN,Chase,04/01/25,4439,04/26/25,93.74,40.0,500,406,100



Transactions:



,date,description,amount,type
0,03/05,UBER *ONE 8005928996 CA,9.99,debit
1,03/11,Payment Thank You-Mobile,159.60,debit
2,03/20,PROSE* PROSE R25320178 PROSE.COM NY,134.94,debit
3,03/02,RALPHS #0294 LOS ANGELES CA,71.75,credit
4,03/05,UBER *ONE HELP.UBER.COM CA,9.99,credit
5,03/06,UBER *EATS HELP.UBER.COM CA,17.67,credit
6,03/05,UBER *ONE HELP.UBER.COM CA,3.00,credit
7,03/18,UBER *EATS HELP.UBER.COM CA,20.46,credit
8,03/20,PROSE* PROSE R25320178 PROSE.COM NY,134.94,credit
9,03/23,COSTCO GAS #0769 INGLEWOOD CA,17.36,credit



Card Info:



,cardholder_name,bank_name,statement_date,account_last4,payment_due_date,new_balance,minimum_payment_due
0,None,Chase,01/06/2025,None,None,None,None



Card Info:



,cardholder_name,bank_name,statement_date,payment_due_date,total_dues,minimum_amount_due,credit_limit,available_credit,available_cash_limit
0,Kalpesh Mehta,HDFC,01/06/2025,21/06/2025,66774.0,3340.0,199000,132226,79600



Transactions:



,date,description,amount,type
0,06/05/2025,TELE TRANSFER CREDIT (Ref# ST25127008300001044...,3463.00,credit
1,10/05/2025,UPI-Chaos control cafe parel,170.00,debit
2,11/05/2025,INNOVATIVERETAILCONCEPTBangalore,296.66,debit
3,12/05/2025,UPI-BOOK MY SHOW,332.04,debit
4,13/05/2025,UPI-PhonePe,1007.80,debit
5,13/05/2025,MUKTA A2 CINEMAS MUMBAI,250.00,debit
6,14/05/2025,TataAIGGeneralInsuranceMumbai,48885.00,debit
7,18/05/2025,J B PATEL CO Mumbai,2020.00,debit
8,21/05/2025,GOLDEN DELICACY MUMBAI,13832.00,debit
